In [2]:
import torch

# torch.get_default_device() 함수가 없다고 떠서 대체 함수 정의


def _get_default_device():
    """torch.get_default_device() 대체 함수"""

    # gpu가 있으면 cuda 디바이스로 반환
    if torch.cuda.is_available():
        return torch.device("cuda")
    # apple m1,m2 칩이 있으면 mps 디바이스로 반환
    elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        return torch.device("mps")
    # 그 외는 cpu 디바이스로 반환
    else:
        return torch.device("cpu")

## 라이브러리 임포트

In [3]:
import os
import torch
import pickle
import copy
from dotenv import load_dotenv
from langchain_core.prompts import load_prompt
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_teddynote import logging
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
import torch.nn.functional as F
import re
from soynlp.normalizer import repeat_normalize
import pandas as pd
import os
import pickle
import copy
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import torch.nn.functional as F
import re
from soynlp.normalizer import repeat_normalize
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from langchain_core.prompts import load_prompt

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# 프로젝트 이름을 입력합니다.
logging.langsmith("chatnge_ai")

llm = ChatOpenAI()

LangSmith 추적을 시작합니다.
[프로젝트명]
chatnge_ai


In [4]:
# transformer 임포트 전에 환경 설정

# transformers 라이브러리가 온라인 모드로 동작하도록 설정
## 모델 다운로드 설정
os.environ["TRANSFORMERS_OFFLINE"] = "0"
# KLUE/BERT 모델 사용하기 위해 Hugging Face 모델 접근할 수 있도록 설정
os.environ["HF_HUB_OFFLINE"] = "0"

In [5]:
try:
    from transformers import AutoTokenizer, AutoModel, logging as transformers_logging

    transformers_logging.set_verbosity_error()  # 경고 메시지 최소화
    print("✅ Transformers 라이브러리 로드 완료")
except ImportError as e:
    print(f"❌ Transformers 라이브러리 임포트 실패: {e}")
    print("pip install transformers를 실행해주세요.")
    raise

✅ Transformers 라이브러리 로드 완료


In [6]:
## torch에 없는 함수 추가
torch.get_default_device = _get_default_device

## 데이터 전처리

In [7]:
# 한국어 불용어 리스트
KOREAN_STOPWORDS = [
    "이",
    "그",
    "저",
    "것",
    "및",
    "에",
    "를",
    "은",
    "는",
    "이런",
    "저런",
    "그런",
    "한",
    "이르",
    "또한",
    "있",
    "하",
    "에서",
    "으로",
    "으로써",
    "로써",
    "로서",
    "로",
    "와",
    "과",
    "이고",
    "이며",
    "이다",
    "있다",
    "하다",
    "되다",
    "이",
    "가",
    "을",
    "를",
    "에게",
    "의",
    "뿐",
    "다",
    "적",
    "데",
    "때",
    "나",
    "도",
    "만",
    "께",
    "에게서",
]

In [8]:
# 한국어 텍스트 전처리
def preprocess_korean_text(text):
    # 만약에 텍스트가 없으면 그냥 빈칸 반환
    if pd.isna(text) or text is None or len(str(text).strip()) == 0:
        return ""

    text = str(text)
    text = re.sub(r"(.)\1{2,}", r"\1\1", text)  # 반복 문자 정규화
    text = re.sub(r"[^가-힣a-zA-Z0-9\s\.,!?]", " ", text)  # 특수문자 제거
    text = re.sub(r"\s+", " ", text).strip()  # 공백 정리

    return text


def remove_stopwords(text, stopwords=KOREAN_STOPWORDS):
    """주어진 텍스트에서 불용어 제거"""
    # 아무것도 없으면 그냥 반환
    if pd.isna(text) or text is None or len(str(text).strip()) == 0:
        return ""

    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]

    return " ".join(filtered_words)

## 감정 분류기 모델

In [9]:
class EmotionClassifier(torch.nn.Module):
    """감정 분류기 모델 - BERT 기반의 감정 분류를 위한 신경망 모델"""

    def __init__(self, bert_model, num_classes, dropout_rate=0.3):
        """
        모델 초기화

        Args:
            bert_model: 사전 훈련된 BERT 모델 (예: BertModel)
            num_classes: 분류할 감정 클래스의 개수
            dropout_rate: 드롭아웃 비율 (기본값: 0.3)
        """
        super(EmotionClassifier, self).__init__()

        # BERT 모델을 백본으로 사용
        self.bert = bert_model
        # BERT의 히든 사이즈 (일반적으로 768)
        self.hidden_size = self.bert.config.hidden_size

        # 과적합 방지를 위한 드롭아웃 레이어
        self.dropout1 = torch.nn.Dropout(dropout_rate)

        # 어텐션 메커니즘: 각 토큰의 중요도를 계산
        self.attention = torch.nn.Sequential(
            # 히든 사이즈를 256차원으로 축소
            torch.nn.Linear(self.hidden_size, 256),
            # Tanh 활성화 함수로 비선형성 추가
            torch.nn.Tanh(),
            # 256차원을 1차원으로 축소하여 어텐션 스코어 생성
            torch.nn.Linear(256, 1),
            # Softmax로 어텐션 가중치를 정규화 (합이 1이 되도록)
            torch.nn.Softmax(dim=1),
        )

        # 최종 분류를 위한 분류기
        self.classifier = torch.nn.Sequential(
            # 히든 사이즈를 256차원으로 축소
            torch.nn.Linear(self.hidden_size, 256),
            # ReLU 활성화 함수
            torch.nn.ReLU(),
            # 배치 정규화로 학습 안정성 향상
            torch.nn.BatchNorm1d(256),
            # 드롭아웃으로 과적합 방지
            torch.nn.Dropout(dropout_rate),
            # 최종 감정 클래스 개수만큼 출력
            torch.nn.Linear(256, num_classes),
        )

    def forward(self, input_ids, attention_mask, token_type_ids):
        """
        순전파 과정

        Args:
            input_ids: 토크나이징된 입력 텍스트의 ID
            attention_mask: 패딩 토큰을 무시하기 위한 마스크
            token_type_ids: 문장 구분을 위한 토큰 타입 ID

        Returns:
            logits: 각 감정 클래스에 대한 점수
        """
        # BERT 모델에 입력을 통과시켜 임베딩 생성
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=True,  # 딕셔너리 형태로 결과 반환
        )

        # 모든 토큰의 히든 상태 (배치 크기, 시퀀스 길이, 히든 크기)
        sequence_output = outputs.last_hidden_state
        # [CLS] 토큰의 풀링된 표현 (배치 크기, 히든 크기)
        pooled_output = outputs.pooler_output

        # 어텐션 가중치 계산: 각 토큰의 중요도 결정
        attention_weights = self.attention(sequence_output)

        # 가중합을 통해 컨텍스트 벡터 생성
        # attention_weights와 sequence_output을 곱하고 시퀀스 차원을 따라 합산
        context_vector = torch.sum(attention_weights * sequence_output, dim=1)

        # 어텐션 기반 컨텍스트 벡터와 BERT의 풀링 출력을 결합
        # 두 표현을 더해서 더 풍부한 표현 생성
        final_output = context_vector + pooled_output

        # 드롭아웃 적용하여 과적합 방지
        final_output = self.dropout1(final_output)

        # 분류기를 통과시켜 최종 로짓 계산
        logits = self.classifier(final_output)

        return logits

## 계층적 감정 분류기

In [ ]:
class HierarchicalEmotionClassifier:
    def __init__(self, model_dir):
        self.model_dir = model_dir  # 모델 경로 설정
        self.device = _get_default_device()  # 디바이스 설정
        self.max_len = 128  # 최대 길이 설정

        # 디버깅용
        print(f"모델 디렉토리: {self.model_dir}")
        print(f"디바이스: {self.device}")

        # 변수 초기화
        self.tokenizer = None
        self.bert_model = None
        self.level1_model = None
        self.level2_model = None
        self.level3_model = None
        self.level1_labels = None
        self.level2_labels = None
        self.level3_labels = None

        # 모델 로드
        self._load_models()

    def _check_files(self):
        print("1단계 : 파일 확인")
        # 필수 파일들 정의
        required_files = [
            "level1_best_model.pt",
            "level2_best_model.pt",
            "level3_best_model.pt",
            "level1_label_encoder.pkl",
            "level2_label_encoder.pkl",
            "level3_label_encoder.pkl",
        ]

        # 없는 파일 확인 리스트
        missing_files = []

        for file in required_files:
            filepath = os.path.join(self.model_dir, file)
            if not os.path.exists(filepath):
                missing_files.append(file)
            else:
                print(f"✅ {file} 파일이 존재합니다.")

        # 만약에 누락된 파일이 있으면 예외 발생
        if missing_files:
            raise FileExistsError(f"파일 누락 목록:{missing_files}")

    # 라벨 인코터 로드
    def _load_label_encoders(self):
        print("2단계: 라벨 인코더 로드")

        encoders = [
            ("level1_label_encoder.pkl", "level1_encoder"),
            ("level2_label_encoder.pkl", "level2_encoder"),
            ("level3_label_encoder.pkl", "level3_encoder"),
        ]

        for filename, attr_name in encoders:
            filepath = os.path.join(self.model_dir, filename)
            with open(filepath, "rb") as f:
                encoder = pickle.load(f)
                setattr(self, attr_name, encoder)
            print(f"{filename} 파일에서 {attr_name} 로드 완료")

    # 3단계 bert 모델 로드
    def _load_bert_model(self):

        try:
            print("3단계: BERT 모델 로드")
            # BERT 모델과 토크나이저 로드
            self.tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
            self.bert_model = AutoModel.from_pretrained("klue/bert-base")

            # 디바이스 이동
            self.bert_model = self.bert_model.to(self.device)
            print("BERT 모델과 토크나이저 로드 완료")
        except Exception as e:
            print(f"❌ BERT 모델 로드 실패: {e}")
            raise ImportError(
                "BERT 모델을 로드하는 데 실패했습니다. 'transformers' 라이브러리가 설치되어 있는지 확인하세요."
            )

    def _load_emotion_models(self):
        """감정 분류 모델들 로드"""
        print("😊 감정 분류 모델 로드 중...")

        models = [
            ("level1_best_model.pt", "level1_model", self.level1_encoder),
            ("level2_best_model.pt", "level2_model", self.level2_encoder),
            ("level3_best_model.pt", "level3_model", self.level3_encoder),
        ]

        for filename, attr_name, encoder in models:
            filepath = os.path.join(self.model_dir, filename)

            print(f"  🔄 {filename} 로드 중...")

            # 모델 초기화
            model = EmotionClassifier(
                copy.deepcopy(self.bert_model.to("cpu")), len(encoder.classes_)
            )

            # 가중치 로드
            try:
                state_dict = torch.load(filepath, map_location="cpu", weights_only=True)
            except TypeError:  # 구버전 PyTorch
                state_dict = torch.load(filepath, map_location="cpu")

            model.load_state_dict(state_dict)

            # 디바이스 이동 및 평가 모드
            model = model.to(self.device)
            model.eval()

            setattr(self, attr_name, model)
            print(f"  ✅ {filename} 로드 완료")

    def _load_models(self):
        """모든 모델 로드"""
        try:
            # 1단계 : 파일 존재 확인
            self._check_files()

            # 2단계 : 라벨 인코더 로드
            self._load_label_encoders()

            # 3단계
            self._load_bert_model()

            # 4단계
            self._load_emotion_models()

        except Exception as e:
            print(f"❌ 모델 로드 중 오류 발생: {e}")
            raise

    # 단일 모델 분류
    def _predict_single(self, model, text):
        # 전처리
        preprocessed = preprocess_korean_text(text)
        preprocessed = remove_stopwords(preprocessed)

        # 토큰화
        encoding = self.tokenizer.encode_plus(
            preprocessed,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )

        # 디바이스 이동
        input_ids = encoding["input_ids"].to(self.device)
        attention_mask = encoding["attention_mask"].to(self.device)
        token_type_ids = encoding["token_type_ids"].to(self.device)

        # 예측
        with torch.no_grad():  # 그래디어언트 계산 비활성화 : 메모리 절약 + 속도 향상
            outputs = model(input_ids, attention_mask, token_type_ids)  # 로짓 출력
            probs = torch.nn.functional.softmax(
                outputs, dim=1
            )  # 확률 변환, 각 클래스별 확률값
            _, preds = torch.max(outputs, dim=1)  # 가장 높은 확률을 가진 클래스 선택

        return preds.item(), probs[0]

    # 계층적 감정 분류
    def predict_hierarchical(self, text):
        """계층적 예측"""
        result = {
            "original_text": text,
            "preprocessed_text": remove_stopwords(preprocess_korean_text(text)),
            "levels": {},
        }

        # 1단계
        pred1, probs1 = self._predict_single(self.level1_model, text)
        label1 = self.level1_encoder.inverse_transform([pred1])[0]

        result["levels"]["level1"] = {
            "step": "1단계: 일반대화 vs 감정",
            "prediction": label1,
            "confidence": float(probs1[pred1]),
            "probabilities": {
                self.level1_encoder.classes_[i]: float(probs1[i])
                for i in range(len(self.level1_encoder.classes_))
            },
        }

        if label1 == "일반대화":
            result["final"] = {
                "prediction": "일반대화",
                "confidence": float(probs1[pred1]),
            }
            result["path"] = ["일반대화"]
            return result

        # 2단계
        pred2, probs2 = self._predict_single(self.level2_model, text)
        label2 = self.level2_encoder.inverse_transform([pred2])[0]

        result["levels"]["level2"] = {
            "step": "2단계: 기쁨 vs 기타감정",
            "prediction": label2,
            "confidence": float(probs2[pred2]),
            "probabilities": {
                self.level2_encoder.classes_[i]: float(probs2[i])
                for i in range(len(self.level2_encoder.classes_))
            },
        }

        if label2 == "기쁨":
            result["final"] = {"prediction": "기쁨", "confidence": float(probs2[pred2])}
            result["path"] = ["감정", "기쁨"]
            return result

        # 3단계
        pred3, probs3 = self._predict_single(self.level3_model, text)
        label3 = self.level3_encoder.inverse_transform([pred3])[0]

        result["levels"]["level3"] = {
            "step": "3단계: 세부 감정 분류",
            "prediction": label3,
            "confidence": float(probs3[pred3]),
            "probabilities": {
                self.level3_encoder.classes_[i]: float(probs3[i])
                for i in range(len(self.level3_encoder.classes_))
            },
        }

        result["final"] = {"prediction": label3, "confidence": float(probs3[pred3])}
        result["path"] = ["감정", "기타감정", label3]

        return result
    
    #임계값 이상의 모든 감정들을 반환
    def predict_with_threshold_emotion(self,text,threshold=0.3):
        result={
            "original_text": text,
            "preprocessed_text": remove_stopwords(preprocess_korean_text(text)),
            "detected_emotions": [],
        }

## 챗봇 생성

In [11]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_core.prompts import MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage


class EmotionChatbot:
    def __init__(self, model_dir):
        # 감정 분류기 초기화
        self.classifier = HierarchicalEmotionClassifier(model_dir)
        try:
            # 챗봇 초기화
            self.llm = ChatOpenAI(
                model="gpt-4o", temperature=0.4, max_tokens=1000, api_key=api_key
            )

            # 메모리 초기화
            self.memory = ConversationSummaryBufferMemory(
                llm=self.llm,
                return_messages=True,
                max_token_limit=2000,
                memory_key="chat_history",
            )

            # 프롬프트 로드
            system_prompt = load_prompt("prompts/emcprompt3.yaml", encoding="utf-8")

            self.chat_prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", system_prompt.template),
                    MessagesPlaceholder(variable_name="chat_history"),
                    ("human", "{message}"),
                ]
            )

            self.chain = self.chat_prompt | self.llm
            print("챗봇 초기화 완료")

        except Exception as e:
            print(f"챗봇 초기화 중 오류 발생: {e}")

    def classify_emotion(self, text):
        # 감정 분류
        return self.classifier.predict_hierarchical(text)

    # 감정 분석
    def format_analysis(self, result):
        """분석 결과 포맷팅"""
        output = f"""
📝 입력: {result['original_text']}
🔍 전처리: {result['preprocessed_text']}

🎯 최종 결과: {result['final']['prediction']} (신뢰도: {result['final']['confidence']:.4f})
📊 예측 경로: {' → '.join(result['path'])}

📈 단계별 분석:
"""

        for level, data in result["levels"].items():
            output += f"\n🔸 {data['step']}\n"
            output += f"   결과: {data['prediction']} ({data['confidence']:.4f})\n"
            output += f"   확률: "
            for emotion, prob in data["probabilities"].items():
                output += f"{emotion}({prob:.3f}) "
            output += "\n"

        return output

    # 챗봇 응답 생성
    # 메모리 추가
    def generate_response(self, text):
        try:
            # 감정 분류
            emotion_result = self.classify_emotion(text)


            #감정 분석 포맷팅
            emotion_info = f"감정: {emotion_result['final']['prediction']}, 신뢰도: {emotion_result['final']['confidence']:.2f}, 경로: {' → '.join(emotion_result['path'])}"


            # 메모리에서 대화 기록 가져오기
            chat_history = self.memory.chat_memory.messages

            # 체인 실행
            response = self.chain.invoke(
                {"message": text, 
                 "emotion_info":emotion_info,
                 "chat_history": chat_history}
            )

            # 응답 추출
            if hasattr(response, "content"):
                ai_response = response.content

                # 메모리에 대화저장
                self.memory.chat_memory.add_user_message(text)
                self.memory.chat_memory.add_ai_message(ai_response)

                return ai_response
            else:
                return "응답생성에 실패했습니다"
        except Exception as e:
            print(f"응답 생성 중 오류 발생: {e}")
            return "응답 생성 중 오류가 발생했습니다. 다시 시도해주세요."

    def process_message(self, text, include_ai_response=True):
        """메시지 전체 처리"""
        # 감정 분석
        emotion_result = self.classify_emotion(text)
        emotion_analysis = self.format_analysis(emotion_result)

        result = {
            "emotion_analysis": emotion_analysis,
            "emotion_result": emotion_result,
        }

        # AI 응답
        if include_ai_response:
            ai_response = self.generate_response(text)
            result["ai_response"] = ai_response
            result["full_response"] = f"{emotion_analysis}\n💬 AI 상담:\n{ai_response}"
        else:
            result["full_response"] = emotion_analysis

        return result

    # 메모리 관련 유틸리티 메서드들
    def get_conversation_summary(self):
        """현재 대화 요약 가져오기"""
        return self.memory.predict_new_summary(self.memory.chat_memory.messages, "")

    def get_chat_history(self):
        """전체 대화 기록 가져오기"""
        return self.memory.chat_memory.messages

    def clear_memory(self):
        """메모리 초기화"""
        self.memory.clear()
        print("💭 대화 기록이 초기화되었습니다.")

    def get_memory_status(self):
        """메모리 상태 확인"""
        messages = self.memory.chat_memory.messages
        token_count = self.memory.llm.get_num_tokens_from_messages(messages)

        return {
            "message_count": len(messages),
            "token_count": token_count,
            "max_token_limit": self.memory.max_token_limit,
            "is_summarizing": token_count > self.memory.max_token_limit,
        }

In [12]:
def create_chatbot(model_dir):
    """챗봇 생성"""
    try:
        return EmotionChatbot(model_dir)
    except Exception as e:
        print(f"❌ 챗봇 생성 실패: {e}")
        return None

In [13]:
# run_chatbot.py - 챗봇 실행

# 1. 챗봇 생성
model_dir = "/Users/hwangeunbi/chatnge_AI/chat/model"  # 모델 파일들이 있는 폴더 경로
chatbot = create_chatbot(model_dir)

if chatbot is None:
    print("챗봇 생성에 실패했습니다.")
    exit()

# 2. 대화 시작
print("💬 심리상담 챗봇이 준비되었습니다!")
print("언제든 'quit' 또는 '종료'를 입력하면 종료됩니다.\n")

while True:
    # 사용자 입력
    user_input = input("당신: ")
    
    # 종료 조건
    if user_input.lower() in ['quit', 'exit', '종료', '끝']:
        print("상담이 종료되었습니다. 좋은 하루 보내세요!")
        break
    
    # 특수 명령어
    if user_input == '/clear':
        chatbot.clear_memory()
        continue
    elif user_input == '/status':
        status = chatbot.get_memory_status()
        print(f"💭 메모리 상태: {status}")
        continue
    
    # 챗봇 응답 생성
    print("\n🤖 상담사가 분석 중...")
    result = chatbot.process_message(user_input, include_ai_response=True)
    
    # 감정 분석 결과 출력 (선택사항)
    print("\n📊 감정 분석:")
    print(result['emotion_analysis'])
    
    # AI 상담 응답 출력
    print(f"\n💬 상담사: {result['ai_response']}")
    print("-" * 50)

모델 디렉토리: /Users/hwangeunbi/chatnge_AI/chat/model
디바이스: mps
1단계 : 파일 확인
✅ level1_best_model.pt 파일이 존재합니다.
✅ level2_best_model.pt 파일이 존재합니다.
✅ level3_best_model.pt 파일이 존재합니다.
✅ level1_label_encoder.pkl 파일이 존재합니다.
✅ level2_label_encoder.pkl 파일이 존재합니다.
✅ level3_label_encoder.pkl 파일이 존재합니다.
2단계: 라벨 인코더 로드
level1_label_encoder.pkl 파일에서 level1_encoder 로드 완료
level2_label_encoder.pkl 파일에서 level2_encoder 로드 완료
level3_label_encoder.pkl 파일에서 level3_encoder 로드 완료
3단계: BERT 모델 로드
BERT 모델과 토크나이저 로드 완료
😊 감정 분류 모델 로드 중...
  🔄 level1_best_model.pt 로드 중...
  ✅ level1_best_model.pt 로드 완료
  🔄 level2_best_model.pt 로드 중...
  ✅ level2_best_model.pt 로드 완료
  🔄 level3_best_model.pt 로드 중...


/var/folders/8n/xfc3hjdn5pq9ch7rw__k4fyh0000gn/T/ipykernel_77496/4267706820.py:17: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationSummaryBufferMemory(


  ✅ level3_best_model.pt 로드 완료
챗봇 초기화 완료
💬 심리상담 챗봇이 준비되었습니다!
언제든 'quit' 또는 '종료'를 입력하면 종료됩니다.


🤖 상담사가 분석 중...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



📊 감정 분석:

📝 입력: 안녕
🔍 전처리: 안녕

🎯 최종 결과: 일반대화 (신뢰도: 1.0000)
📊 예측 경로: 일반대화

📈 단계별 분석:

🔸 1단계: 일반대화 vs 감정
   결과: 일반대화 (1.0000)
   확률: 감정(0.000) 일반대화(1.000) 


💬 상담사: 안녕하세요. 이렇게 찾아와 주셔서 고마워요. 오늘 어떤 이야기를 나누고 싶으신가요?
--------------------------------------------------

🤖 상담사가 분석 중...

📊 감정 분석:

📝 입력: 아 너무 그냥 짜증나고 다 때려치고 싶어
🔍 전처리: 아 너무 그냥 짜증나고 때려치고 싶어

🎯 최종 결과: 분노 (신뢰도: 0.9623)
📊 예측 경로: 감정 → 기타감정 → 분노

📈 단계별 분석:

🔸 1단계: 일반대화 vs 감정
   결과: 감정 (1.0000)
   확률: 감정(1.000) 일반대화(0.000) 

🔸 2단계: 기쁨 vs 기타감정
   결과: 기타감정 (0.9999)
   확률: 기쁨(0.000) 기타감정(1.000) 

🔸 3단계: 세부 감정 분류
   결과: 분노 (0.9623)
   확률: 당황(0.010) 분노(0.962) 불안(0.011) 상처(0.008) 슬픔(0.009) 


💬 상담사: 그렇게 느끼는 건 정말 힘들 수 있겠네요. 무언가가 당신을 많이 지치게 하고 있는 것 같아요. 혹시 어떤 상황이나 일이 그런 감정을 불러일으키고 있는지 조금 더 이야기해 줄 수 있을까요?
--------------------------------------------------

🤖 상담사가 분석 중...

📊 감정 분석:

📝 입력: 그냥 취업도 잘 안되고 되는게 없어
🔍 전처리: 그냥 취업도 잘 안되고 되는게 없어

🎯 최종 결과: 불안 (신뢰도: 0.4865)
📊 예측 경로: 감정 → 기타감정 → 불안

📈 단계별 분석:

🔸 1단계: 일반대화 vs 감정
   결과: 감정 (1.0000)
   확률: 감정(1.000)